In [88]:
import os
import numpy as np
import re
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *-?\ *[0-9]+)?')


In [89]:
'''
this is a program for calculating the time derivatives of the veloctity field. 

as this isn't supported by openfoam post-processsing, we'll start with a simple method of just delta_u/delta_t

we'll read in the lists of values for the uniform mesh at each timestep and then subtract the previous timestep's ones

'''

"\nthis is a program for calculating the time derivatives of the veloctity field. \n\nas this isn't supported by openfoam post-processsing, we'll start with a simple method of just delta_u/delta_t\n\nwe'll read in the lists of values for the uniform mesh at each timestep and then subtract the previous timestep's ones\n\n"

In [90]:
def getU(t):
    '''
    for a given timestep, this method extracts the velocity data from the openfoam file
    returns the velocity data in a numpy array, where each entry is the velocity in a cell
    
    THIS METHOD IS DESIGNED TO WORK WITH THE 2D SYSTEMS.  WILL NEED SOME MODIFICATION FOR THE FILE LAYOUT IN THE 3D ONES
    '''
    t_string = "{:.1f}".format(t)
    
    #removes trailing 0 and decimal point to be consistent with openfoam
    #directory naming structure
    if t_string[-1] == '0':
        t_string = t_string[:-2]
        
    filename = t_string+'/U'
    internal_mesh = []
    outlet_boundary = []
    
    with open(filename) as u_file:
        
        while True:
            line = u_file.readline()
            
            if line.startswith('internalField'):
                break
                
        
        next(u_file)
        next(u_file)
        
        #this section gets the values of the internal mesh cells
        while(True):
            line = u_file.readline()
            if "boundaryField" in line:
                break
            else:
                #this is a rather hacky way of dealing with the closing bracket of the internal mesh
                #list of vectors
                if len(line) > 2:
                    #this finds all numbers enclosed in brackets
                    internal_mesh.append(re.search(r'\((.*?)\)',line).group(1))
                    
        
        #this section gets the values of the rear boundary
        while(True):
            
            line = u_file.readline()
            if "rightWall" in line:
                #skips to the relevant values
                next(u_file)
                next(u_file)
                next(u_file)
                boundary_line = u_file.readline()
                #use a regex to get a list of all the cell vector values at the boundary
                boundary_values = re.findall('\(.*?\)',boundary_line) #all of the values at the boundary
                boundary_cellVals = [re.findall(match_number, u) for u in boundary_values if re.findall(match_number, u)] #vectors of each of the cells
                outlet_boundary = [[float(u) for u in v] for v in boundary_cellVals]
                break
            
    #print((u_list))
        
    for i in range(len(internal_mesh)-1, -1, -1):
        
        internal_mesh[i] = internal_mesh[i].strip().split(' ')
        internal_mesh[i] = [re.findall(match_number, u)[0] for u in internal_mesh[i] if re.findall(match_number, u)]
        internal_mesh[i] = [float(u) for u in internal_mesh[i] if u]
        
        if internal_mesh[i] == []:
            del internal_mesh[i]

    
    internal_mesh_array = np.array([np.array(u) for u in internal_mesh])
    outlet_boundary_array = np.array([np.array(u) for u in outlet_boundary])
    
    return internal_mesh_array, outlet_boundary_array

In [91]:
def calc_dUdt(t, delta_t):
    '''this method returns a crude approximation of the time derivatives delta u/ delta t
    
    returns an array with the differences
    '''
    
    t2 = t
    t1 = max(0, t2-delta_t)
    u2_internal, u2_boundary = getU(t2)
    u1_internal, u1_boundary = getU(t1)
    
    dUdt_internal = (u2_internal - u1_internal)/delta_t
    dUdt_boundary = (u2_boundary - u1_boundary)/delta_t

    return dUdt_internal, dUdt_boundary

In [150]:
def write_dUdt(t, delta_t):
    '''
    this method writes the calculated time derivatives to an OpenFOAM friendly format with the 
    hope that this will allow them to be read into paraview.
    
    This method is for the 2D systems, it will need some slight modification for the boundary naming
    systems of the 3D ones.
    '''
    t_string = "{:.1f}".format(t)
    #removes trailing 0 and decimal point to be consistent with openfoam
    #directory naming structure
    if t_string[-1] == '0':
        t_string = t_string[:-2]
        
    dUdt_internal, dUdt_boundary = calc_dUdt(t, delta_t)
        
    filename = t_string+'/dUdt'
        
    with open(filename, 'w+') as file:
        
        file.write("/*--------------------------------*- C++ -*----------------------------------*\\\n")
        file.write("  =========                 |\n")
        file.write("  \\\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox\n")
        file.write("   \\\    /   O peration     | Website:  https://openfoam.org\n")
        file.write("    \\\  /    A nd           | Version:  7\n")
        file.write("     \\\/     M anipulation  |\n")
        file.write("\\*---------------------------------------------------------------------------*/\n")
        file.write("FoamFile\n")
        file.write("{\n")
        file.write("version     2.0;\n")
        file.write("format      ascii;\n")    
        file.write("class       volVectorField;\n")    
        file.write('location    "'+t_string+'";\n')
        file.write("object      dUdt;\n")
        file.write("}\n")
        file.write("// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //\n")   
        
        file.write("\ndimensions      [0 1 -2 0 0 0 0];\n")
        
        file.write("\ninternalField   nonuniform List<vector>\n")
        file.write(str(len(dUdt_internal))+"\n")
        file.write("(\n")
        
        for i in range(len(dUdt_internal)):
            file.write('({:.5e} {:.5e} {:.5e})\n'.format(dUdt_internal[i][0], dUdt_internal[i][1], dUdt_internal[i][2]))
        
        file.write(")\n")
        file.write(";\n")
        
        file.write("\nboundaryField\n")
        file.write("{\n")
        
        file.write("\tleftWall\n")
        file.write("\t{\n")
        file.write("\t\ttype \t\t zeroGradient;\n")
        file.write("\t}\n")
        
        file.write("\trightWall\n")
        file.write("\t{")
        file.write("\t\ttype \t\t inletOutlet;\n")
        file.write("\t\tinletValue \t uniform (0 0 0);\n")
        file.write("\t\tvalue \t\t nonuniform List<vector>\n")
        file.write(str(len(dUdt_boundary))+"\n")
        file.write("(\n")
        for i in range(len(dUdt_boundary)):
            file.write('({:.5e} {:.5e} {:.5e})\n'.format(dUdt_boundary[i][0], dUdt_boundary[i][1], dUdt_boundary[i][2]))
        file.write(")\n")
        file.write(";\n")
        file.write("\t}\n")
        
        file.write("\tlowerWall\n")
        file.write("\t{\n")
        file.write("\t\ttype \t\t noSlip;\n")
        file.write("\t}\n")
        
        file.write("\tupperWall\n")
        file.write("\t{\n")
        file.write("\t\ttype \t\t fixedValue;\n")
        file.write("\t\tvalue \t\t uniform (0 0 0);\n") #TIME DERIVATIVE AT TOP OF SYSTEM IS ZERO DUE TO CONSTANT VELOCITY B.C.
        file.write("\t}\n")
        
        file.write("\tdefaultFaces\n")
        file.write("{\n")
        file.write("\t\ttype \t\t empty;\n")
        file.write("\t}\n")
        
        file.write("}\n\n")
        file.write("// ************************************************************************* //")
        
            
        
    

In [151]:
write_dUdt(2, 0.2)

In [156]:
t_max = 30
delta_t = 0.1

for t in np.arange(0.2, t_max+delta_t, delta_t):
    print(t)
    write_dUdt(t, delta_t)

0.2
0.30000000000000004
0.4000000000000001
0.5000000000000001
0.6000000000000001
0.7000000000000002
0.8000000000000003
0.9000000000000001
1.0000000000000002
1.1000000000000003
1.2000000000000004
1.3000000000000003
1.4000000000000004
1.5000000000000004
1.6000000000000003
1.7000000000000004
1.8000000000000005
1.9000000000000006
2.000000000000001
2.1000000000000005
2.200000000000001
2.3000000000000007
2.400000000000001
2.500000000000001
2.600000000000001
2.700000000000001
2.800000000000001
2.9000000000000012
3.000000000000001
3.100000000000001
3.200000000000001
3.300000000000001
3.4000000000000012
3.5000000000000013
3.6000000000000014
3.7000000000000015
3.8000000000000016
3.9000000000000012
4.000000000000001
4.100000000000001
4.200000000000002
4.300000000000002
4.400000000000001
4.500000000000002
4.600000000000001
4.700000000000002
4.800000000000002
4.900000000000002
5.000000000000002
5.100000000000001
5.200000000000002
5.300000000000002
5.400000000000002
5.500000000000002
5.6000000000000